In [ ]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from PIL import Image
from PIL import ImageDraw, ImageFont
from utils.utils import cvtColor
from utils.dataloader import YoloDataset
from yolo import YOLO


In [ ]:
dataset_path = './dataset/apples/'
anchors_path = 'model_data/yolo_anchors.txt'
model_path = 'saved_models/ep048-loss9.262-val_loss5.039.pth'
classes_path = 'model_data/my_classes.txt'
    
num_classes = 2
input_shape = [416, 416]
anchors_mask = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
with open(dataset_path + 'sets/train.txt', 'r') as file:
    train_splits = file.read().splitlines()
with open(dataset_path + 'sets/val.txt', 'r') as file:
    val_splits = file.read().splitlines()    
print(len(train_splits), len(val_splits))
yolo_model = YOLO(model_path = model_path, classes_path = classes_path)

In [ ]:
def draw_image(image, boxes):
    for i, box in enumerate(boxes):
        top, left, bottom, right = box
        top     = max(0, np.floor(top).astype('int32'))
        left    = max(0, np.floor(left).astype('int32'))
        bottom  = min(image.size[1], np.floor(bottom).astype('int32'))
        right   = min(image.size[0], np.floor(right).astype('int32'))

        draw = ImageDraw.Draw(image)
        draw.rectangle([left, top, right, bottom])
        del draw
    return image           


In [ ]:
#for split in train_splits:
split = val_splits[5]
image = Image.open(dataset_path + 'images/' + split + '.png')
image = cvtColor(image)
print(image.size)
with open(dataset_path + 'annotations/' + split+ '.csv', 'r') as file:
    annotations = np.loadtxt(file, delimiter = ',', skiprows = 1)
boxes = np.zeros([len(annotations), 4])
for i, anno in enumerate(annotations):
    boxes[i, 0] = anno[2] - anno[3]
    boxes[i, 1] = anno[1] - anno[3]
    boxes[i, 2] = anno[2] + anno[3]
    boxes[i, 3] = anno[1] + anno[3]   

In [ ]:
draw_image(image, boxes)

In [ ]:
image = Image.open(dataset_path + 'images/' + split + '.png')
new_image, outputs = yolo_model.detect_image(image)

In [ ]:
new_image